In [14]:
// This question was asked in Fidelity Investments interview for Data Engineer (Senior role).

// It was a LeetCode SQL problem (Hard level) -- 𝗛𝘂𝗺𝗮𝗻 𝗧𝗿𝗮𝗳𝗳𝗶𝗰 𝗼𝗳 𝗦𝘁𝗮𝗱𝗶𝘂𝗺.

// ❓ Display the records with three or more rows with consecutive id's, and the number of people is greater than or equal to 100 for each.

// 👉 Table Column - id (int), visit_date (date), people (int)

// 𝗜𝗻𝗽𝘂𝘁:
// +------+------------+-----------+
// | id   | visit_date | people    |
// +------+------------+-----------+
// | 1    | 2017-01-01 | 10        |
// | 2    | 2017-01-02 | 109       |
// | 3    | 2017-01-03 | 150       |
// | 4    | 2017-01-04 | 99        |
// | 5    | 2017-01-05 | 145       |
// | 6    | 2017-01-06 | 1455      |
// | 7    | 2017-01-07 | 199       |
// | 8    | 2017-01-09 | 188       |
// +------+------------+-----------+


// 𝗢𝘂𝘁𝗽𝘂𝘁:
// +------+------------+-----------+
// | id   |visit_date  |   people  |
// +------+------------+-----------+
// | 5    | 2017-01-05 |   145     |
// | 6    | 2017-01-06 |   1455    |
// | 7    | 2017-01-07 |   199     |
// | 8    | 2017-01-09 |   188     |
// +------+------------+-----------+


val visitors=Seq(
    (1,"2017-01-01",10), 
    (2,"2017-01-02",109), 
    (3,"2017-01-03",150), 
    (4,"2017-01-04",99), 
    (5,"2017-01-05",145), 
    (6,"2017-01-06",1455), 
    (7,"2017-01-07",199), 
    (8,"2017-01-09",188)
).toDF("id","visit_date","people").withColumn("visit_date", to_date($"visit_date", "yyyy-MM-dd"))

visitors.show(false)
visitors.printSchema()

+---+----------+------+
|id |visit_date|people|
+---+----------+------+
|1  |2017-01-01|10    |
|2  |2017-01-02|109   |
|3  |2017-01-03|150   |
|4  |2017-01-04|99    |
|5  |2017-01-05|145   |
|6  |2017-01-06|1455  |
|7  |2017-01-07|199   |
|8  |2017-01-09|188   |
+---+----------+------+

root
 |-- id: integer (nullable = false)
 |-- visit_date: date (nullable = true)
 |-- people: integer (nullable = false)



visitors: org.apache.spark.sql.DataFrame = [id: int, visit_date: date ... 1 more field]


In [27]:
visitors.createOrReplaceTempView("visitors_tbl")


// +---+----------+------+---+----+-------+
// |id |visit_date|people|rn |diff|counts |
// +---+----------+------+---+----+-------+
// |2  |2017-01-02|109   |1  |1   |2      |
// |3  |2017-01-03|150   |2  |1   |2      |
// |5  |2017-01-05|145   |3  |2   |4      |
// |6  |2017-01-06|1455  |4  |2   |4      |
// |7  |2017-01-07|199   |5  |2   |4      |
// |8  |2017-01-09|188   |6  |2   |4      |
// +---+----------+------+---+----+-------+

val data = spark.sql("""
WITH CTE AS (
    SELECT
        *,
        id-row_number() over(order by visit_date) diff
    FROM visitors_tbl
    WHERE people>=100
),
CTE2 AS (
SELECT 
    *,
    COUNT(id) over(partition by diff order by diff) counts 
FROM CTE
)
SELECT * FROM CTE2 WHERE counts >= 3
""")

data.show(false)


+---+----------+------+----+------+
|id |visit_date|people|diff|counts|
+---+----------+------+----+------+
|5  |2017-01-05|145   |2   |4     |
|6  |2017-01-06|1455  |2   |4     |
|7  |2017-01-07|199   |2   |4     |
|8  |2017-01-09|188   |2   |4     |
+---+----------+------+----+------+



data: org.apache.spark.sql.DataFrame = [id: int, visit_date: date ... 3 more fields]


In [24]:
data.drop("diff","rnk").show(false)

+---+----------+------+
|id |visit_date|people|
+---+----------+------+
|5  |2017-01-05|145   |
|6  |2017-01-06|1455  |
|7  |2017-01-07|199   |
|8  |2017-01-09|188   |
+---+----------+------+

